# Load

In [10]:
#from ydata_profiling import ProfileReport
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from data import aggregate_daily_records

daily_records = aggregate_daily_records()
daily_records = daily_records.sort_values(by='date')
daily_records = daily_records.reset_index()
daily_records[daily_records['date'] == '2023-08-24']

,index,date,daily_records
235,235,2023-08-24,191


In [ ]:
df = pd.read_csv('files/timeseries_haul_loading_data.csv')
df.dtypes

truck                  object
loader                 object
ton                   float64
n_shovel              float64
truck_total_cycle     float64
loader_total_cycle    float64
distance_empty        float64
distance_full         float64
date                   object
dtype: object

In [315]:
df['date'] = pd.to_datetime(df['date'])
df['truck'] = df['truck'].astype('category')
df['loader'] = df['loader'].astype('category')

In [316]:
df.isna().sum()

truck                 0
loader                0
ton                   0
n_shovel              0
truck_total_cycle     0
loader_total_cycle    0
distance_empty        0
distance_full         0
date                  0
dtype: int64

## Registros vs tiempo

Primero analizaremos la cantidad total de datos disponibles 

In [317]:
daily_records = df.groupby('date').size().reset_index(name='records')
monthly_records = df
monthly_records['month'] = monthly_records['date'].dt.to_period('M')
monthly_records = monthly_records.groupby('month').size().reset_index(name='records')

In [318]:
daily_records = df.groupby('date').size().reset_index(name='daily_records')
daily_records['month'] = daily_records['date'].dt.to_period('M')
monthly_avg = daily_records[['month','daily_records']].groupby('month')['daily_records'].agg(
    month_mean='mean',
    month_median='median',
    month_q1=lambda x: x.quantile(0.25),
    month_q3=lambda x: x.quantile(0.75),
    month_iqr=lambda x: x.quantile(0.75) - x.quantile(0.25),
    month_std = 'std'
).reset_index()
monthly_avg = monthly_avg.set_index('month')

daily_records = daily_records.join(monthly_avg, on='month')

In [319]:
px.histogram(daily_records, x='daily_records', title='Distribución de registros diarios')

En general la cantidad de registros diarios presenta una distribución similar a una normal, sin embargo, cabe analizar cómo varía esto en el tiempo.

In [320]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=daily_records['date'], y=daily_records['daily_records'], mode='lines', name='Registros diarios')
)
fig.add_trace(
    go.Scatter(x=daily_records['date'], y=daily_records['month_mean'], mode='lines', name='Promedio mensual')
)
fig.update_layout(title='Cantidad de registros diarios', xaxis_title='Fecha', yaxis_title='Registros')
fig.show()


Notamos que la cantidad de registros totales aumenta con el tiempo. Es una tendencia anual y no parece ser una variación periódica o estacional.

Además detectamos días problemáticos, la cantidad diaria de registros presenta más outliers por límite inferior que superior.

In [321]:
daily_records_copy = daily_records.copy()
daily_records_copy['month'] = daily_records_copy['month'].astype(str)
fig = px.violin(daily_records_copy, x='month', y='daily_records', box=True,
                title="Monthly Distribution of Daily Records",
                labels={'month': 'Month', 'records': 'Records per Day'})

fig.show()

In [322]:
normalized_daily_records = daily_records
normalized_daily_records['daily_records'] = (normalized_daily_records['daily_records'] - normalized_daily_records['daily_records'].mean()) / normalized_daily_records['daily_records'].std()
px.histogram(normalized_daily_records, x='daily_records', title='Normalized distribution of daily records')

In [323]:
lower_daily_records_outliers = daily_records[daily_records['daily_records'] < 400]

## Registros por camión

In [324]:
truck = df.groupby('truck',observed=False).size().reset_index(name='records')
truck = truck.sort_values(by='records', ascending=False)
truck_loader = df.groupby(['truck', 'loader'], observed=False).size().reset_index(name='records')
truck_loader = truck_loader.sort_values(by='records', ascending=False)

In [325]:
px.bar(truck, x='truck', y='records', title='Cantidad de registros por camión')

Se detecta que un camión cuenta con una cantidad mucho menor que el resto de camiones. Analizaremos en detalle su comportamiento.

In [ ]:
from typing import Tuple
import pandas as pd

df = pd.read_csv('files/timeseries_haul_loading_data.csv')
df['date'] = pd.to_datetime(df['date'])
trucks_daily_cycles = df[['truck', 'date']].sort_values(by='truck')

trucks_first_date = trucks_daily_cycles.groupby('truck', observed=True)['date'].min()
trucks_first_date = trucks_first_date.reset_index(name='first_date').set_index('truck')

trucks_daily_cycles = trucks_daily_cycles.join(trucks_first_date, on='truck')
trucks_daily_cycles['date'] = trucks_daily_cycles['date']-trucks_daily_cycles['first_date']
trucks_daily_cycles['date'] = trucks_daily_cycles['date'].dt.days
trucks_daily_cycles = trucks_daily_cycles.drop(columns='first_date')

all_trucks = trucks_daily_cycles.groupby(['truck', 'date'], observed=False).size().reset_index(name='daily_cycles')
CAEX_61 = all_trucks[all_trucks['truck'] == 'CAEX61']
CAEX_61 = CAEX_61[['date', 'daily_cycles']]
CAEX_61.columns = ['days_since_first_cycle', 'daily_cycles']
CAEX_61 = CAEX_61.sort_values(by='days_since_first_cycle')

other_trucks = all_trucks[all_trucks['truck'] != 'CAEX61']
other_trucks = other_trucks[['date', 'daily_cycles']].groupby('date', observed=False)
other_trucks = other_trucks.agg(
    mean = ('daily_cycles', 'mean'),
    median = ('daily_cycles', 'median'),
    q1 = ('daily_cycles', lambda x: x.quantile(0.25)),
    q3 = ('daily_cycles', lambda x: x.quantile(0.75)),
    min = ('daily_cycles', 'min'),
).reset_index(names='days_since_first_cycle')


#other_trucks_daily_cycles = trucks_daily_cycles[trucks_daily_cycles['truck'] != 'CAEX61']
#other_trucks_avg_daily_cycles =other_trucks_daily_cycles.groupby('date', observed=False)['daily_cycles'].mean().reset_index()

CAEX_61

,truck,date,daily_cycles
13837,CAEX61,0,3
13838,CAEX61,1,8
13839,CAEX61,2,6
13840,CAEX61,3,8
13841,CAEX61,4,3
...,...,...,...
13907,CAEX61,78,2
13908,CAEX61,79,9
13909,CAEX61,80,21
13910,CAEX61,81,6


In [327]:
fig = go.Figure()


# Agregar la línea del promedio de los otros camiones
fig.add_trace(
    go.Scatter(
        x=other_trucks_avg_daily_records['date'],
        y=other_trucks_avg_daily_records['avg_daily_records'],
        name='Promedio de registros diarios de otros camiones',
        )
        )

# Agregar los registros diarios de CAEX_61
fig.add_trace(
    go.Scatter(
        x=CAEX_61_daily_records['date'], 
        y=CAEX_61_daily_records['daily_records'],
        name='Registros diarios de CAEX_61')
        )

# Configurar el título y las etiquetas
fig.update_layout(title='Cantidad de registros',
                  xaxis_title='Días desde el primer registro del camión',
                  yaxis_title='Registros diarios')

fig.show()

Tiene un periodo de inactividad claro, pero incluso antes de quedar inactivo presentaba muchos días con pocos registros. Nos interesa entonces analizar cuándo un camión está con pocos registros, cuál podría ser un threshold.

In [328]:
px.box(trucks_daily_records, x='truck', y='daily_records', 
             title="Daily Records Distribution per Truck (Boxplot)",
             labels={'truck': 'Truck', 'daily_records': 'Daily Records'})

Se aprecia que varios camiones tienen un Q1 bajo (<3 cargas diarias).
Además se detecta que 

In [329]:
trucks_daily_records_q1 = trucks_daily_records.groupby('truck', observed=False)['daily_records'].quantile(0.25).reset_index(name='q1')
max_daily_records_q1 = int(trucks_daily_records_q1['q1'].max())+1

fig = px.histogram(
    trucks_daily_records_q1, 
    x='q1', 
    nbins=max_daily_records_q1, 
    title='Q1 de registros diarios por camión', 
    labels={'q1': 'Q1'})
fig.show()

Se ve una clara concentración del Q1 entre 6 y 10

In [330]:
import plotly.express as px

# Calcular el porcentaje de días inactivos para cada umbral y añadirlo a la lista
trucks_total_days = trucks_daily_records.groupby('truck', observed=False)['date'].max()
trucks_total_days = trucks_total_days.reset_index(name='total_days').set_index('truck')

# Crear una lista de barras para cada umbral y luego el gráfico
for inactive_days_threshold in [0, 1, 2, 3]:
    trucks_inactive_days = trucks_daily_records[trucks_daily_records['daily_records'] <= inactive_days_threshold]
    trucks_inactive_days = trucks_inactive_days.groupby('truck', observed=False)['daily_records'].count()
    trucks_inactive_days = trucks_inactive_days.reset_index(name='inactive_days').set_index('truck')

    trucks_inactive_days = trucks_inactive_days.join(trucks_total_days)
    trucks_inactive_days['inactive_days_percentage'] = trucks_inactive_days['inactive_days'] / trucks_inactive_days['total_days']
    trucks_inactive_days = trucks_inactive_days.sort_values(by='inactive_days_percentage', ascending=False)

    # Crear gráfico
    fig = px.bar(
        trucks_inactive_days, 
        x=trucks_inactive_days.index, 
        y='inactive_days_percentage', 
        title=f'Porcentaje de días inactivos (umbral de inactividad: {inactive_days_threshold} días)', 
        labels={'index': 'Camión', 'inactive_days_percentage': 'Porcentaje de días inactivos'}
    )

    # Añadir el toggle para mostrar/ocultar 'CAEX_61'
    fig.update_layout(
        updatemenus=[
            {
                'buttons': [
                    {
                        'args': [{'visible': [True if truck != 'CAEX_61' else False for truck in trucks_inactive_days.index]}],
                        'label': 'Ocultar CAEX_61',
                        'method': 'update'
                    },
                    {
                        'args': [{'visible': [True] * len(trucks_inactive_days)}],
                        'label': 'Mostrar Todos',
                        'method': 'update'
                    }
                ],
                'direction': 'down',
                'showactive': True,
                'active': 0,
                'x': 0.17,
                'xanchor': 'left',
                'y': 1.15,
                'yanchor': 'top'
            }
        ]
    )

    # Ajustar los límites del eje y dinámicamente dependiendo de si 'CAEX_61' está visible o no
    y_max = trucks_inactive_days['inactive_days_percentage'].max()
    fig.update_yaxes(range=[0, y_max + 0.05])  # Ajuste de límite superior con margen de 5%

    fig.show()


En general, variar el threshold de inactividad entre 0 y 3 no produce diferencias significativas. Un threshold lo suficientemente alto podría agrupar a muchos camiones con CAEX61, sin embargo, en el rango explorado este se diferencia significativamente del resto, por lo que podemos considerarlo como un caso aparte del resto en término de registros diarios.

Queda entonces analizar otras variables como la cantidad de carga transportada por los camiones.